### Проект. Анна Запорощенко, Софья Генералова

### Предобработка текстов

Вам нужно собрать свой корпус текстов (100 штук не менее 100 слов каждый / 10 000 словоупотреблений). Корпус состоит из текстов одного жанра или тематики (не отрывки одного текста).

Можно делать это вручную или скачивая html-страницы средствами Питона (например, библиотека requests). Затем нужно удалить нетекстовые элементы, разделить корпус на предложения. Не забудьте, что для каждого предложения нужно помнить, из какого источника оно берется.


In [ ]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=9b56653bc6810549f781278a699b7834490ec96ec6fa5ccd1e17822b41b446d4
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [34]:
import urllib.request
from fake_useragent import UserAgent
import re
import random

In [2]:
# функция возвращает html код страницы
def get_html_content(url):
    user_agent = UserAgent().chrome
    req = urllib.request.Request(url, headers={'User-Agent':user_agent})
    with urllib.request.urlopen(req) as response:
        html_content = response.read().decode('utf-8')
    return html_content

In [3]:
# функция возвращает ссылки на новости, собранные на странице
def get_links(html_content, pattern, links):
    l = re.findall(pattern, html_content)
    links.extend(l)
    return links

In [4]:
# инициализируем список для ссылок
# генерируем номера страниц со ссылками на новости (первая без номера, ее оставим) и достаем ссылки из них
links = []
pages = [t for t in range(2,7)]
pattern = '<a href=\'(.+?)\'><h3 class="news-big"'
for n in pages:
    url = 'https://liveberlin.ru/news/'
    url = url + 'page/' + str(n)
    links = get_links(get_html_content(url), pattern, links)

In [5]:
# функция убирает нерасшифровавшиеся знаки, рекламу, теги в строке
def clear(line):
    line = line.replace('&nbsp;', ' ')
    line = line.replace('&#8217;', "'")
    line = line.replace('\xa0', ' ')
    line = re.split('<.+?>', line)
    line = ('').join(line)
    line = line.replace(' Реклама в «Живом Берлине» | liveberlin.ad@gmail.com', '')
    line = re.sub('\s+', ' ',line) # убирает повторяющиеся пробельные символы
    return line

In [6]:
# функция достает из html кода новостной страницы текст новости, ее автора и заголовок
def get_info(html_content):
    text = clear((' ').join(re.findall('<p>.+?</p>', html_content)[:-2]))
    title = clear(('').join(re.findall("<meta property=\'og:type\' content=\'article\' />\n    <meta property=\'og:title\' content=\'(.+?)' />", html_content)))
#     author= re.findall("title=\".+?\" rel=\"author\">(.+?)</a>&nbsp;", html_content)[0]
    return text, title #, author

In [7]:
texts = []
titles = []
# authors = []

In [8]:
for link in links:
    html_content = get_html_content(link)
#     text, title, author 
    text, title = get_info(html_content)
    texts.append(text)
    titles.append(title + '\n' + link)
#     authors.append(author)

In [29]:
titles

['Ограничениям скорости на немецких автобанах не бывать!\nhttps://liveberlin.ru/news/2019/11/04/ogranicheniyam-skorosti-na-nemetskih-avtobanah-ne-byvat/',
 'Берлинский транспорт опоздал? Получи компенсацию! Вскоре и онлайн\nhttps://liveberlin.ru/news/2019/10/31/berlinskij-transport-opozdal-poluchi-kompensatsiyu-vskore-i-onlajn/',
 'Регламент о защите данных (DSGVO/GDPR) заработал в Германии в полную силу. И это очень жестко\nhttps://liveberlin.ru/news/2019/10/30/reglament-o-zaschite-dannyh-dsgvo-gdpr-zarabotal-v-germanii-v-polnuyu-silu-i-eto-ochen-zhestko/',
 'Продукты из конопли на полках немецких магазинов: легально, но не пасторально\nhttps://liveberlin.ru/news/2019/10/27/produkty-iz-konopli-na-polkah-nemetskih-magazinov-legalno-no-ne-pastoralno/',
 'Берлинский общественный транспорт подорожает с начала 2020 года. Но не без вариантов\nhttps://liveberlin.ru/news/2019/10/06/berlinskij-obschestvennyj-transport-podorozhaet-s-nachala-2020-goda-no-ne-bez-variantov/',
 'Граждане ЕС смогут 

### Морфологический анализ текстов

Здесь можно использовать любой инструмент / библиотеку, которая позволяет проводить морфологический анализ.
Подумайте, нужно ли вам разрешение морфологической неоднозначности. Хорошо бы сказать об этом в презентации и обосновать выбранное решение.
Не забудьте, что для запросов тоже понадобится морфологический анализ.

Natasha делит по предложениям.

In [112]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

In [14]:
segmenter = Segmenter()

emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)

morph_vocab = MorphVocab()

In [43]:
next_start = 0
for text, text_meta in zip(texts, titles):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    next_start = sent_a(doc, text_meta, next_start) # след начальный айди предложения
    
    

сделать ф-ю: sent_a(doc, text_meta, start=0)

In [35]:
def sent_a(doc, text_meta, start=0):
    next_start = 0
    for sent_id, sent in enumerate(doc.sents, start):
        next_start = sent_id
        sent_text = sent.text
#         print(sent_text)
        sentence = []
        for tok in sent.tokens:
            pos = tok.pos
            if pos != 'PUNCT':
                tok.lemmatize(morph_vocab)
                lem = tok.lemma
                tex = tok.text
                sentence.append(tex + '<pos:' + pos + '>' + '<lem:' + lem + '>')
        # ДЕЛИТ НА ЭНГРАММЫ
        sent_n_gramm(sentence, sent_text, sent_id, text_meta)
        
    return next_start+1

In [46]:
big_dict

{'Партия зеленых предложила': {'sent_id': [0],
  'text_meta': ['Ограничениям скорости на немецких автобанах не бывать!\nhttps://liveberlin.ru/news/2019/11/04/ogranicheniyam-skorosti-na-nemetskih-avtobanah-ne-byvat/'],
  'sent_text': ['Партия зеленых предложила ввести ограничение скорости 130 км/ч на немецких автобанах, с целью сократить количество выхлопных газов.'],
  'tag_ngr': ['Партия<pos:NOUN><lem:партия> зеленых<pos:ADJ><lem:зеленый> предложила<pos:VERB><lem:предложить>']},
 'зеленых предложила ввести': {'sent_id': [0],
  'text_meta': ['Ограничениям скорости на немецких автобанах не бывать!\nhttps://liveberlin.ru/news/2019/11/04/ogranicheniyam-skorosti-na-nemetskih-avtobanah-ne-byvat/'],
  'sent_text': ['Партия зеленых предложила ввести ограничение скорости 130 км/ч на немецких автобанах, с целью сократить количество выхлопных газов.'],
  'tag_ngr': ['зеленых<pos:ADJ><lem:зеленый> предложила<pos:VERB><lem:предложить> ввести<pos:VERB><lem:ввести>']},
 'предложила ввести ограничени

13782 3gramm

In [42]:
big_dict = {}

In [41]:
def sent_n_gramm(mas_for_sent, sent_text, sent_id, text_meta):
    rr = []
    ln = len(mas_for_sent)
    if ln < 3 and ln > 0:
        rr.append(' '.join(mas_for_sent))
    else:
        for i in range(ln):
            if i < ln-2:
                res = mas_for_sent[i] + ' ' + mas_for_sent[i+1] + ' ' + mas_for_sent[i+2]
                rr.append(res)
                
    for n_gr in rr:
        ws = n_gr.split('> ')
        ngrm = [] # list from which i will concatenate strs
        for w in ws: # для кажд части энграммы
            w_text = re.search('([^<>]+)<', w).group(1)
            ngrm.append(w_text)
    
        ngrm_s = ' '.join(ngrm)
        if ngrm_s not in big_dict:
            big_dict[ngrm_s] = {'sent_id': [], 'text_meta':[], 'sent_text':[], 'tag_ngr':[]}
            
        if sent_id not in big_dict[ngrm_s]['sent_id'] or n_gr not in big_dict[ngrm_s]['tag_ngr']:
            big_dict[ngrm_s]['sent_id'].append(sent_id)
            big_dict[ngrm_s]['text_meta'].append(text_meta)
            big_dict[ngrm_s]['sent_text'].append(sent_text)
            big_dict[ngrm_s]['tag_ngr'].append(n_gr)
            

In [49]:
import sqlite3

In [73]:
#подключаемся к базе данных
conn = sqlite3.connect('project_db.db')

In [74]:
cur = conn.cursor()

таблица для предложений

In [75]:
cur.execute("CREATE TABLE IF NOT EXISTS 'Sentences'('sent_id' integer PRIMARY KEY AUTOINCREMENT, 'title_link', 'sent_orig')")

таблица энграмм

In [76]:
cur.execute("CREATE TABLE IF NOT EXISTS 'Corpus'('ngr_id' integer PRIMARY KEY AUTOINCREMENT, '3-gram0_orig', '3-gram1_orig', '3-gram2_orig', '3-gram0_lem', '3-gram1_lem', '3-gram2_lem', '3-gram0_pos', '3-gram1_pos', '3-gram2_pos', 'sent' integer, FOREIGN KEY('sent') REFERENCES 'Sentences'('sent_id'))")

In [77]:
conn.commit()
conn.close()

из словаря big_dict все записывается в бд

In [78]:
conn = sqlite3.connect('project_db.db')
c = conn.cursor()

In [79]:
countt = 0 # счетчик для энграмм
for gr in big_dict:
    for i in range(len(big_dict[gr]['sent_id'])):
        sent_id = big_dict[gr]['sent_id'][i]
            
        title_link = big_dict[gr]['text_meta'][i]
        sent_orig = big_dict[gr]['sent_text'][i]
        ngr_id = big_dict[gr]['tag_ngr'][i]
        
        ws = ngr_id.split('> ')
        
        # могут быть не только триграммы, но и биграммы, униграммы (теоретически - если предложение состояло из 1-2 слов)
        
        n_gram1_orig, n_gram1_lem, n_gram1_pos = '', '', ''
        n_gram2_orig, n_gram2_lem, n_gram2_pos = '', '', ''
        
        for j in range(len(ws)):
            w_text = re.search('([^<>]+)<', ws[j]).group(1)
            pos = re.search('<pos:([^>]+)>', ws[j]).group(1)
            lem = re.search('<lem:([^>]+)>?', ws[j]).group(1)
            if j == 0:
                n_gram0_orig = w_text
                n_gram0_lem = lem
                n_gram0_pos = pos
            elif j == 1:
                n_gram1_orig = w_text
                n_gram1_lem = lem
                n_gram1_pos = pos
            else:
                n_gram2_orig = w_text
                n_gram2_lem = lem
                n_gram2_pos = pos
        
        c.execute('INSERT OR IGNORE INTO "Sentences" VALUES (?, ?, ?)', (sent_id, title_link, sent_orig)) 
        # если попробует добавить с повт праймари ки, то проигнорит
        c.execute('INSERT INTO "Corpus" VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (countt, n_gram0_orig, n_gram1_orig, n_gram2_orig, n_gram0_lem, n_gram1_lem, n_gram2_lem, n_gram0_pos, n_gram1_pos, n_gram2_pos, sent_id))
        conn.commit()
        countt += 1
        

In [80]:
conn.commit()
conn.close()

### указать какой набор тегов / стандарт разметки вы используете. Иногда достаточно ссылки на библиотеку (например, тегсет Mystem)

### функция для поиска

поиск - нужно найти все предложения, где слово встречается в любой форме
“поиска” - нужно найти предложения только с этой формой
знать+NOUN - нужно найти все предложения, где встречается существительное “знать”
NOUN - найти все предложения с существительными

В питоне это, скорее всего, будет выглядеть так:
search(‘поиск’)
search(‘“поиск”’) - дублируем кавычки
search(‘знать+NOUN’)
search(‘NOUN VERB ADVB’)
search(‘ADJ дом’)

Запрос состоит из последовательных слов/POS-тегов (n-грамма,  максимум 3-грамма), к каждому применяются правила выше.

In [132]:
list_of_pos = set(['ADJ',
'ADV',
'DET',
'AUX',
'CONJ',
'VERB',
'INTJ',
'NOUN',
'PRON',
'NUM',
'PRCL',
'PART',
'ADP']) # тег 'PUNCT' будет считаться словом

In [109]:
def generate_request(input_s):
    res_words = []
    
    words = input_s.split(' ') # кошек "нет" NOUN
    for w_index, word in enumerate(words):
        if word[0] == '"':
            if word[-1] != '"': # если кавычка не закрыта
                res_words = []
                print('Попробуйте закрыть двойные кавычки')
                return res_words
            
            #поиск по форме слова
            line = "Corpus.'3-gram@_orig' = " + "'" + word.strip('"') + "'"
        elif word in list_of_pos:
            #поиск по POS
            line = "Corpus.'3-gram@_pos' = " + "'" + word + "'"
        elif '+' in word:
            if word[0] == '+' or word[-1] == '+': # проверкa 'кошка+', 'noun' or '+кошка', 'noun' or '+'
                res_words = []
                print('знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN')
                return res_words
            
            #поиск по POS и леммам. если плюс в середине чего-то напр bbb+aaa
            lemma, POS = word.split('+')
            
            # проверка на то что pos это pos
            if POS not in list_of_pos:
                res_words = []
                print('знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN')
                return res_words
        
            # проверка на то что lemma это лемма # кошкой+NOUN
            # лемматизируем наташей
            lem = get_lem(word)
                
            line = "Corpus.'3-gram@_lem' = " + "'" + lem + "'" + " AND Corpus.'3-gram@_pos' = " + POS
            
        else:
            lem = get_lem(word)
            #поиск по леммам
            line = "Corpus.'3-gram@_lem' = " + "'" + lem + "'"
        
        res_words.append(line)

    return res_words

In [9]:
# lemmatization
def get_lem(word): # на вход одно слово
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)   
    doc.tokens[0].lemmatize(morph_vocab) 
    # если сегментер разделит по дефису то возьмется лемма 1й половины
    lem = doc.tokens[0].lemma
    
    return lem

In [113]:
# норм запрос
generate_request('кошек "нет" NOUN')

["Corpus.'3-gram@_lem' = 'кошка'",
 "Corpus.'3-gram@_orig' = 'нет'",
 "Corpus.'3-gram@_pos' = 'NOUN'"]

In [114]:
# не норм запрос
generate_request('коше+к "нет" NOUN')

знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN


[]

In [115]:
# ок запрос
generate_request('кошек "не+т" NOUN')

["Corpus.'3-gram@_lem' = 'кошка'",
 "Corpus.'3-gram@_orig' = 'не+т'",
 "Corpus.'3-gram@_pos' = 'NOUN'"]

In [129]:
# незакрытые кавычки
generate_request('кошек "нет NOUN')

Попробуйте закрыть двойные кавычки


[]

In [28]:
import re

In [105]:
def search(input_s):
    input_s = re.sub('\s+', ' ', input_s).strip() # замена повт пробелов на один
    check = len(input_s.split(' '))
    if check > 3 or check < 1:
        return 'Запрос некорректен'
    
    words = generate_request(input_s)
    if words == []:
        return 'Запрос некорректен'
        
    if check == 1:
        request = one_word_requests(words)
        print_scentences(request)
    elif check == 2:
        request = two_words_requests(words)
        print_scentences(request)
    elif check == 3:
        request = three_words_requests(words)
        print_scentences(request)

In [97]:
def extract_sents_id(case, c): # на вход id sents
    for el in case:
        req = "SELECT sent_orig, title_link FROM Sentences WHERE Sentences.'sent_id' = " + str(el[0])
        sent_orig, title_link = c.execute(req).fetchone()
        print('SENT: ' + sent_orig + '\n' + 'TITLE+URL: ' + title_link)
        print()

In [125]:
def print_scentences(request):
    conn = sqlite3.connect('project_db.db')
    c = conn.cursor()
    case = c.execute(request).fetchall()
    if case == []:
        print('По вашему запросу было найдено НИЧЕГО')
    else:
        extract_sents_id(case, c)
    c.close()
    conn.close()

In [73]:
def one_word_requests(words): # на вход list, ключи индексы 0, 1 или 2
    one_w_req = words[0]
    requests = []
    for index in range(3):
        ending = re.sub('@', str(index), one_w_req)
        requests.append(ending)
    res = 'SELECT DISTINCT sent FROM Corpus WHERE ' + ' OR '.join(requests) 
    # distinct чтобы без повторов
    
    return res # возвращает 1 запрос

пример получаемого запроса в базу

In [131]:
one_word_requests(generate_request('кошек'))

"SELECT DISTINCT sent FROM Corpus WHERE Corpus.'3-gram0_lem' = 'кошка' OR Corpus.'3-gram1_lem' = 'кошка' OR Corpus.'3-gram2_lem' = 'кошка'"

In [46]:
def two_words_requests(words):
    requests = []
    indexes = [[0, 1], [1, 2]]
    for pair in indexes:
#         beginning = 'SELECT DISTINCT sent FROM Corpus WHERE '
        middle = re.sub('@', str(pair[0]), words[0])
        conj = ' AND '
        ending = re.sub('@', str(pair[1]), words[1])
        requests.append('(' + middle + conj + ending + ')')
    
    res = 'SELECT DISTINCT sent FROM Corpus WHERE ' + ' OR '.join(requests) 
    return res

пример получаемого запроса в базу

In [47]:
two_words_requests(generate_request('кошек NOUN'))

"SELECT DISTINCT sent FROM Corpus WHERE (Corpus.'3-gram0_lem' = 'кошка' AND Corpus.'3-gram1_pos' = 'NOUN') OR (Corpus.'3-gram1_lem' = 'кошка' AND Corpus.'3-gram2_pos' = 'NOUN')"

In [50]:
def three_words_requests(words):
#     requests = []
#     indexes = [0, 1, 2]
    beginning = 'SELECT DISTINCT sent FROM Corpus WHERE '
    middle_first = re.sub('@', '0', words[0])
    middle_second = re.sub('@', '1', words[1])
    conj = ' AND '
    ending = re.sub('@', '2', words[2])
    res = beginning + middle_first + conj + middle_second + conj + ending
    
    return res

пример получаемого запроса в базу

In [51]:
three_words_requests(generate_request('кошек NOUN поить+VERB'))

"SELECT DISTINCT sent FROM Corpus WHERE Corpus.'3-gram0_lem' = 'кошка' AND Corpus.'3-gram1_pos' = 'NOUN' AND Corpus.'3-gram2_lem' = 'поить' AND Corpus.'3-gram2_pos' = VERB"

# запуск!!!!!!!!!!!!!!!

### примеры запросов
Приведите примеры запросов, которые можно задавать к вашему корпусу. 

(Ещё можно описать тематику, чтобы проверяющий мог придумать свои запросы)

In [111]:
search('"как дела"')

Попробуйте закрыть двойные кавычки


'Запрос некорректен'

In [117]:
search('"Партия"')

SENT: Партия зеленых предложила ввести ограничение скорости 130 км/ч на немецких автобанах, с целью сократить количество выхлопных газов.
TITLE+URL: Ограничениям скорости на немецких автобанах не бывать!
https://liveberlin.ru/news/2019/11/04/ogranicheniyam-skorosti-na-nemetskih-avtobanah-ne-byvat/

SENT: По опросам, в Британии лидирует созданная в начале апреля этого года «Партия Brexit», занимающая крайне жесткую позицию в поддержку выхода страны из союза.
TITLE+URL: Великобритания примет участие в выборах в Европарламент несмотря на Brexit
https://liveberlin.ru/news/2019/05/09/velikobritaniya-primet-uchastie-v-vyborah-v-evroparlament-nesmotrya-na-brexit/

SENT: Партия этих рыбешек прибыла в начале марта из Гамбурга, куда они в свою очередь были доставлены с Атлантического побережья Франции, сообщает Berliner Morgenpost.
TITLE+URL: Экология: В берлинские реки выпустят два миллиона угрей
https://liveberlin.ru/news/2019/03/19/ekologiya-v-berlinskie-reki-vypustyat-dva-milliona-ugrej/



In [127]:
search('"Партия" VERB ADV')

По вашему запросу было найдено НИЧЕГО


### бонусы

Бонус - если вы придумали какие-то оптимизации, чтобы поиск работал быстрее, не забудьте сказать о них :)

Ещё бонус - разбор ошибок, которые делает функция поиска


### Если jupyter-тетрадка, то можно ссылку на гитхаб.
